# Chapter 33: OS and Platform

This notebook covers Python's `os`, `os.path`, `platform`, and `sys` modules for interacting with the operating system. You will learn how to inspect environment variables, manipulate file paths, traverse directory trees, and query system information.

## Key Concepts
- **`os.environ`**: A mapping of environment variables
- **`os.path`**: Portable path manipulation (join, split, basename, dirname, splitext)
- **`os.getcwd()`** / **`os.listdir()`**: Working directory and directory listing
- **`os.walk()`**: Recursive directory tree traversal
- **`platform`**: System identification (OS name, architecture, Python version)
- **`sys`**: Interpreter-level information (version, path, executable)

## Section 1: Working Directory and Directory Listing

`os.getcwd()` returns the current working directory as a string. `os.listdir()` returns the names of entries in a given directory.

In [ ]:
import os

# Get the current working directory
cwd: str = os.getcwd()
print(f"Current working directory: {cwd}")
print(f"Is a directory: {os.path.isdir(cwd)}")

# List entries in the current directory
entries: list[str] = os.listdir(cwd)
print(f"\nNumber of entries: {len(entries)}")
print(f"First 5 entries: {sorted(entries)[:5]}")

## Section 2: Environment Variables with `os.environ`

`os.environ` is a mapping object representing the environment. You can read variables with bracket access or `.get()`, and set them by assignment.

In [ ]:
import os

# os.environ behaves like a dictionary
print(f"Type: {type(os.environ)}")
print(f"'PATH' in environ: {'PATH' in os.environ}")

# Read an environment variable safely with .get()
home: str | None = os.environ.get("HOME")
print(f"HOME: {home}")

# Read with a default for missing variables
editor: str = os.environ.get("EDITOR", "not set")
print(f"EDITOR: {editor}")

# List a few environment variable names
env_keys: list[str] = sorted(os.environ.keys())
print(f"\nSample env vars: {env_keys[:5]}")

In [ ]:
import os

# Set and read a custom environment variable
os.environ["MY_APP_MODE"] = "testing"
print(f"MY_APP_MODE: {os.environ['MY_APP_MODE']}")

# Remove it when done
del os.environ["MY_APP_MODE"]
print(f"After delete: {os.environ.get('MY_APP_MODE', 'not found')}")

## Section 3: Path Manipulation with `os.path`

`os.path` provides functions for portable path manipulation. These work on path strings without requiring the path to actually exist on disk.

In [ ]:
import os

# Join path components portably
path: str = os.path.join("src", "chapter_33", "__init__.py")
print(f"Joined path: {path}")

# Extract the base name (last component)
base: str = os.path.basename(path)
print(f"Basename: {base}")

# Extract the directory name (everything except last component)
directory: str = os.path.dirname(path)
print(f"Dirname: {directory}")

# Split into (directory, basename)
head, tail = os.path.split(path)
print(f"\nsplit -> head: {head!r}, tail: {tail!r}")

In [ ]:
import os

# Split name and extension
name, ext = os.path.splitext("script.py")
print(f"Name: {name!r}, Extension: {ext!r}")

# Works with full paths too
name2, ext2 = os.path.splitext("/home/user/data.tar.gz")
print(f"Name: {name2!r}, Extension: {ext2!r}")

# Absolute path and normalization
rel: str = os.path.join("src", "..", "tests")
normalized: str = os.path.normpath(rel)
absolute: str = os.path.abspath(rel)
print(f"\nRelative:   {rel}")
print(f"Normalized: {normalized}")
print(f"Absolute:   {absolute}")

In [ ]:
import os

# Check if paths exist and their types
cwd: str = os.getcwd()

print(f"exists('{cwd}'): {os.path.exists(cwd)}")
print(f"isdir('{cwd}'):  {os.path.isdir(cwd)}")
print(f"isfile('{cwd}'): {os.path.isfile(cwd)}")

# Check a non-existent path
fake: str = os.path.join(cwd, "does_not_exist.txt")
print(f"\nexists(fake): {os.path.exists(fake)}")
print(f"isabs(fake):  {os.path.isabs(fake)}")

## Section 4: Creating and Removing Directories

`os.makedirs()` creates directories (including parents), and `os.rmdir()` removes empty directories.

In [ ]:
import os
import tempfile

# Use a temporary directory so we don't litter the filesystem
with tempfile.TemporaryDirectory() as tmpdir:
    # Create nested directories in one call
    nested: str = os.path.join(tmpdir, "level1", "level2", "level3")
    os.makedirs(nested)
    print(f"Created: {nested}")
    print(f"Exists:  {os.path.isdir(nested)}")

    # exist_ok=True prevents errors if directory already exists
    os.makedirs(nested, exist_ok=True)
    print("makedirs with exist_ok=True succeeded")

    # Remove the innermost empty directory
    os.rmdir(nested)
    print(f"\nAfter rmdir, exists: {os.path.isdir(nested)}")

print(f"\nTempdir cleaned up: {not os.path.exists(tmpdir)}")

## Section 5: Walking Directory Trees with `os.walk()`

`os.walk()` generates `(dirpath, dirnames, filenames)` tuples for every directory in a tree. It is the standard way to recursively process a file system hierarchy.

In [ ]:
import os
import tempfile

# Build a small directory tree
with tempfile.TemporaryDirectory() as tmpdir:
    os.makedirs(os.path.join(tmpdir, "sub"))
    open(os.path.join(tmpdir, "file.txt"), "w").close()
    open(os.path.join(tmpdir, "sub", "nested.txt"), "w").close()

    # Walk the tree
    dirs_found: list[str] = []
    files_found: list[str] = []

    for dirpath, dirnames, filenames in os.walk(tmpdir):
        dirs_found.append(dirpath)
        for fname in filenames:
            full_path: str = os.path.join(dirpath, fname)
            files_found.append(full_path)
        print(f"Directory: {os.path.relpath(dirpath, tmpdir)}")
        print(f"  Subdirs:  {dirnames}")
        print(f"  Files:    {filenames}")

    print(f"\nTotal directories visited: {len(dirs_found)}")
    print(f"Total files found: {len(files_found)}")

In [ ]:
import os
import tempfile

def find_files_by_extension(root: str, extension: str) -> list[str]:
    """Find all files with a given extension under root."""
    matches: list[str] = []
    for dirpath, dirnames, filenames in os.walk(root):
        for fname in filenames:
            if fname.endswith(extension):
                matches.append(os.path.join(dirpath, fname))
    return matches

# Demonstrate with a temporary tree
with tempfile.TemporaryDirectory() as tmpdir:
    # Create test files
    for name in ["app.py", "utils.py", "readme.md", "data.csv"]:
        open(os.path.join(tmpdir, name), "w").close()
    os.makedirs(os.path.join(tmpdir, "pkg"))
    open(os.path.join(tmpdir, "pkg", "models.py"), "w").close()

    py_files: list[str] = find_files_by_extension(tmpdir, ".py")
    print(f"Python files found: {len(py_files)}")
    for f in py_files:
        print(f"  {os.path.relpath(f, tmpdir)}")

## Section 6: Platform and System Information

The `platform` module provides detailed information about the operating system, hardware, and Python interpreter.

In [ ]:
import platform

# Operating system identification
print(f"System:     {platform.system()}")       # e.g., 'Darwin', 'Linux', 'Windows'
print(f"Release:    {platform.release()}")
print(f"Version:    {platform.version()}")
print(f"Machine:    {platform.machine()}")       # e.g., 'x86_64', 'arm64'
print(f"Processor:  {platform.processor()}")
print(f"Platform:   {platform.platform()}")

In [ ]:
import platform
import sys

# Python version information
print(f"Python version (platform):  {platform.python_version()}")
print(f"Python version (sys):       {sys.version}")

# sys.version_info gives structured access
vi = sys.version_info
print(f"\nVersion info: major={vi.major}, minor={vi.minor}, micro={vi.micro}")

# Verify they agree
expected: str = f"{vi.major}.{vi.minor}.{vi.micro}"
matches: bool = platform.python_version() == expected
print(f"platform matches sys: {matches}")

## Section 7: The `sys` Module

The `sys` module provides access to Python interpreter internals -- the module search path, the executable path, and more.

In [ ]:
import sys

# Python executable and prefix
print(f"Executable: {sys.executable}")
print(f"Prefix:     {sys.prefix}")
print(f"Platform:   {sys.platform}")    # e.g., 'darwin', 'linux', 'win32'

# Module search path (first few entries)
print(f"\nModule search path (sys.path):")
for i, p in enumerate(sys.path[:5]):
    print(f"  [{i}] {p}")
if len(sys.path) > 5:
    print(f"  ... ({len(sys.path)} total entries)")

In [ ]:
import sys

# Integer and recursion limits
print(f"Max integer size (for indexing): {sys.maxsize}")
print(f"Recursion limit: {sys.getrecursionlimit()}")
print(f"Byte order: {sys.byteorder}")

# Size of objects in bytes
sample_int: int = 42
sample_str: str = "hello"
sample_list: list[int] = [1, 2, 3]

print(f"\nSize of int 42:      {sys.getsizeof(sample_int)} bytes")
print(f"Size of str 'hello': {sys.getsizeof(sample_str)} bytes")
print(f"Size of list [1,2,3]: {sys.getsizeof(sample_list)} bytes")

## Section 8: Practical Pattern -- System Info Report

Combining `os`, `platform`, and `sys` to build a system information report that could be useful for debugging or logging.

In [ ]:
import os
import platform
import sys


def system_info_report() -> dict[str, str]:
    """Collect system information into a dictionary."""
    return {
        "os_name": platform.system(),
        "os_release": platform.release(),
        "machine": platform.machine(),
        "python_version": platform.python_version(),
        "python_executable": sys.executable,
        "cwd": os.getcwd(),
        "home": os.environ.get("HOME", os.environ.get("USERPROFILE", "unknown")),
    }


report: dict[str, str] = system_info_report()
print("System Information Report")
print("=" * 40)
for key, value in report.items():
    print(f"{key:>20}: {value}")

## Summary

### `os` Module
- **`os.getcwd()`**: Returns the current working directory
- **`os.listdir(path)`**: Lists entries in a directory
- **`os.environ`**: Dictionary-like mapping of environment variables
- **`os.makedirs(path, exist_ok=True)`**: Creates nested directories
- **`os.walk(top)`**: Recursively yields `(dirpath, dirnames, filenames)` tuples

### `os.path` Module
- **`join()`**: Combines path components portably
- **`basename()` / `dirname()`**: Extracts the last component or everything before it
- **`splitext()`**: Splits filename and extension
- **`exists()` / `isdir()` / `isfile()`**: Tests for path existence and type
- **`abspath()` / `normpath()`**: Converts to absolute or normalized form

### `platform` Module
- **`system()`**: OS name (`'Darwin'`, `'Linux'`, `'Windows'`)
- **`machine()`**: Hardware architecture (`'x86_64'`, `'arm64'`)
- **`python_version()`**: Python version string

### `sys` Module
- **`sys.executable`**: Path to the Python interpreter
- **`sys.path`**: Module search path list
- **`sys.version_info`**: Structured version information
- **`sys.getsizeof()`**: Memory size of an object in bytes